<a href="https://colab.research.google.com/github/noora20FH/heart_disease_classification/blob/main/heart_disease_classification_spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Source https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease 

In [2]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=b01e2d004f6000c6fa61cf3f70c25303bf524c849177b3dbd84e30aaf2fd4e5c
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [4]:
from google.colab import drive


#Access my google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np

In [14]:
#imports and setup
from pyspark.sql import SparkSession
from pyspark.ml.feature import (VectorAssembler, OneHotEncoder, StringIndexer)
from pyspark.ml import Pipeline
from pyspark.ml.classification import (LogisticRegression, RandomForestClassifier, NaiveBayes)
from pyspark.sql.functions import (col, explode, array, lit)
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, MulticlassClassificationEvaluator)
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F
 
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
 
spark = SparkSession.builder.appName('HeartDiseaseClassification').getOrCreate()



In [19]:
heartdf = pd.read_csv (r'/content/drive/MyDrive/TUGAS_NOORA/heart_2020_cleaned.csv',header=None)

print ("Original Dataset (Rows:Colums): ")
print (heartdf.shape)

heartdf.loc[:5,:]

Original Dataset (Rows:Colums): 
(319796, 18)


<ipython-input-19-d3353d9ce140>:1: DtypeWarning: Columns (1,5,6,14) have mixed types. Specify dtype option on import or set low_memory=False.
  heartdf = pd.read_csv (r'/content/drive/MyDrive/TUGAS_NOORA/heart_2020_cleaned.csv',header=None)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
1,No,16.6,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
2,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
3,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
4,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
5,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [15]:
read_file = pd.read_csv (r'/content/drive/MyDrive/TUGAS_NOORA/heart_2020_cleaned.csv', header=True)
# df = spark.read.csv(r'/content/drive/MyDrive/TUGAS_NOORA/heart_2020_cleaned.csv',inferSchema=True,header=True)
display(read_file.head(5))

TypeError: ignored